## Импорт нужных модулей

In [1]:
import requests
from datetime import datetime
import sqlite3
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from pymystem3 import Mystem
m = Mystem()
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from tqdm.auto import tqdm
import re

## Нужные данные
- ``token`` — токен приложения;
- ``another_token`` — токен пользователя (не было на паре, но он нужен для поиска пользователей с определённым критерием, например, родным городом).

In [2]:
token = '???'
version = '5.92'
another_token = '!!!'

## База данных
- подключение;
- создание нужных таблиц

In [3]:
conn = sqlite3.connect('my_vk_new.db')
cur = conn.cursor()

In [4]:
cur.execute("""
CREATE TABLE IF NOT EXISTS users 
(user_id int PRIMARY KEY, name text, surname text, sex text, bdate text, town text)
""")
cur.execute("""
CREATE TABLE IF NOT EXISTS posts 
(post_id text PRIMARY KEY, post_text text, post_lemmas text, smiles text, date text, likes int, user_id int)
""")
conn.commit()

## Функция для лемматизирования и очистки тектса
Ещё она делает список использованных смайликов, а сами смайлики выкидывает из текста, но почему-то работает не для всех символов, нужно смотреть таблицы дальше.

In [5]:
def lemmas(text):
    emojis = []
    tokens = word_tokenize(text)
    tokens = [i for i in tokens if (i not in string.punctuation)]
    stop_words = stopwords.words('russian')
    tokens = [i for i in tokens if (i not in stop_words)]
    two_text = ' '.join(tokens)
    two_text = two_text.replace('.', '')
    lemmas = m.lemmatize(two_text)
    new_text = ''.join(lemmas[:-1])
    for l in lemmas:
            emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"
            u"\U0001F300-\U0001F5FF"
            u"\U0001F680-\U0001F6FF"
            u"\U0001F1E0-\U0001F1FF"
            u"\U0001F6D0-\U0001FAD6"
            u"\U00012600-\U000126CF"
            u"\U000126D0-\U0001F25D"
                               "]+", flags=re.UNICODE)
            emo = re.match(emoji_pattern, l)
            if not emo == None:
                emojis.append(emo.group())
                new_text = new_text.replace(emo.group(), '')
    new_text = re.sub(r'  +', ' ', new_text)
    my_emojis = ''.join(emojis)
    return new_text, my_emojis

## Функция, которая получает 10 постов, и для каждого из них:
- находит текст поста;
- находит лемматизированный текст;
- находит дату написания;
- получает число лайков на посте;
- записывает это всё в базу, если текст — не пустой.

Возвращает текст и число записанных постов.

При этом удовлетворительными признаются посты, в которых 35 или более слов.

In [6]:
def find_posts(user_id):
    my_conn = sqlite3.connect('my_vk_new.db')
    cur = my_conn.cursor()
    one_count = 0
    text = None
    my_data = requests.get(
    'https://api.vk.com/method/wall.get',
    params={
        'owner_id': user_id,
        'access_token': token,
        'v': version,
        'count': 100
        }).json()
    if 'response' in my_data.keys():
        for i in my_data['response']['items']:
            if len(i['text'].split(' ')) >= 35:
                one_count += 1
                text = i['text']
                lemmatized, emoj = lemmas(text)
                my_post_id = str(i['owner_id'])+'_'+str(i['id'])
                date = i['date']
                my_date = datetime.fromtimestamp(date)
                my_likes  = i['likes']['count']              
                my_conn = sqlite3.connect('my_vk_new.db')
                cur = my_conn.cursor()
            
                sqlite_insert_with_param = """INSERT INTO posts
                                  (post_id, post_text, post_lemmas, smiles, date, likes, user_id)
                                  VALUES (?, ?, ?, ?, ?, ?, ?);"""

                data_tuple = (my_post_id, text, lemmatized,emoj, my_date, my_likes, user_id)
                cur.execute(sqlite_insert_with_param, data_tuple)
                my_conn.commit()
            if one_count >= 10:
                break
    return(text, one_count)

## Функция, которая ищет пользователей с возрастом от заданного числа и:
- определяет их имя и фамилию;
- пол;
- дату рождения, если указано;
- родной город, если указано;

И записывает это в базу данных, если у них есть хотя бы один подходящий пост.

In [7]:
def find_users(from_age, to_age, count):
    my_conn = sqlite3.connect('my_vk_new.db')
    cur = my_conn.cursor()
    two_count = 0
    data = requests.get(
        'https://api.vk.com/method/users.search',
        params={
            'age_from': from_age,
            'age_to': to_age,
            'fields': 'home_town, sex, bdate',
            'access_token': another_token,
            'v': version,
            'count': count
        }
    ).json()
    for d in tqdm(data['response']['items']):
        my_user_id = d['id']
        t, c = find_posts(my_user_id)
        two_count += c
        my_name = d['first_name']
        my_surname = d['last_name']
        s = d['sex']
        if s == 1:
            my_sex = 'Женский'
        elif s == 2:
            my_sex = 'Мужской'
        else:
            my_sex = 'Другой'
        if 'bdate' in d.keys():
            bday = d['bdate']
        else:
            bday = 'no'
        if 'home_town' in d.keys() and d['home_town'] != '':
            town = d['home_town']
        else:
            town = 'no'
        if t:
            print(my_name, my_surname)
            print(c)           
            sqlite_insert_with_param = """INSERT INTO users
                              (user_id, name, surname, sex, bdate, town)
                              VALUES (?, ?, ?, ?, ?, ?);"""

            data_tuple = (my_user_id, my_name, my_surname, my_sex, bday, town)
            cur.execute(sqlite_insert_with_param, data_tuple)
            my_conn.commit()
            print(two_count)
        if two_count >= 1000:
            my_conn.close()
            break

In [8]:
find_users(14, 20, 1000)

Виталий Чувашев
2
2
Данил Плужников
10
12
Владимир Голубь
1
13
Артём Васильев
2
15
Азер Насибов
5
20
Аня Кулиева
4
24
Илья Сецко
2
26
Луиза-Габриэла Бровина
5
31
Artur Babich
1
32
Александра Набатчикова
2
34
Егор Насакин
2
36
Михаил Чолокоглы
10
46
Ира Сидоркова
10
56
Максим Покатаев
10
66
Юля Дерюшева
1
67
Никита Кадников
2
69
Кристина Кошевая
2
71
Ксюша Карпова
10
81
Георгий Молчанов
1
82
Лина Каширина
3
85
Fedor Salomatov
2
87
Артур Сопельник
3
90
Даниил Шевченко
8
98
Юлия Ляшева
2
100
Алиса Кожикина
10
110
Максим Типси
6
116
Лия Шамсина
10
126
Алексей Ченский
9
135
Дмитрий Грищенко
5
140
Даниил Авдышев
9
149
Боб Джек
10
159
Анастасия Князева
2
161
Федя Басс
2
163
Полина Зиновьева
10
173
Мария Воробьева
3
176
Миша Пунтов
10
186
Наталия Костенева
9
195
Вика Старикова
10
205
Дмитрий Куплинов
5
210
Герман Гласко
10
220
Ильяс Гаязов
2
222
Максим Жаткин
2
224
Стас Орлов
8
232
Кристина Пакарина
2
234
Дмитрий Китель
10
244
Андрей Чернышев
2
246
Влад Глент
1
247
Валентина Ляпина
10
257
Катя